<a href="https://colab.research.google.com/github/cs-pub-ro/ML/blob/master/lab2/Laborator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Învățare Automată
# Grupare. Algoritmul K-Means
### Autori:
* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului

Scopul laboratorului îl reprezintă întelegerea conceptului de arbore de decizie și implementarea unor clasificatori bazați pe acest model.

## 2. Problema de rezolvat

Problema de rezolvat ı̂n acest laborator este una de ı̂nvățare supervizată: fiind dat un **set de date X** ce conține exemple descrise printr-un set de **atribute discrete A** și etichetate cu **câte o clasă dintr-o mulțime cunoscută C**, să se construiască un model pentru clasificarea exemplelor noi.

## 3. Arbore de decizie


Un arbore de decizie este un clasificator ce aproximează funcții discrete.

Într-un arbore de decizie există 2 tipuri de noduri:
* *noduri intermediare* - conține un test pentru un atribut și are câte un arc (și implicit un subarbore) pentru fiecare valoare posibiliă a atributului
* *noduri frunză* - este etichetat cu o clasă

Pentru **a clasifica un obiect nou** se pornește din rădăcina arborelui și din fiecare nod se coboară pe arcul corespunzător valorii atributului pe care o are obiectul dat. Atunci când se ajunge ı̂ntr-un nod frunză, clasa acestuia va reprezenta predicția arborelui.

## 4. Păduri de arbori aleatori

*Pădurile de arbori aleatori* (eng. Random Forest) este un model format din mai mulți arbori de decizie.

Se bazează pe 2 hiperparametrii:
* Eșantionare aleatoare din setul de date de antrenament
* Subseturi aleatoare de atribute considerate la împărțirea pe mai multi subarbori

Predicția, utilizând un astfel de model, se bazează pe clasa majoritară oferită de predicțiile indepente ale tuturor arborilor.

## 5. Workspace Setup

### Câteva biblioteci de care vom avea nevoie

In [0]:
from math import log2
import csv

### Hiperparametrii necesari rulării

In [0]:
DATASET_NAME = 'Car'  #@param ['Chess', 'Car', 'Tennis']

# Adâncimea arborilor
D = 3 #@param {type: "slider", min: 2, max: 10}

# Procentul de exemple din setul de date utilizat la construcția arborilor
P = 50 #@param {type: "slider", min: 1, max: 100}

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [0]:
class Node:
    """ Representation for a node from the decision tree """
    def __init__(self, label):
        """
            for non-leafs it is the name of the attribute
            for leafs it is the class
        """
        self.label = label
        
        """ Keep mapping (attribute_value, node)
            All the nodes from childre[attr_val] are having the attr_val as
            an attribute
        """
        self.children = {}
    
    def display(self, string):
        print(string + self.label)
        string += "\t"
        if self.children:
            for key, value in self.children.items():
                print(string + key)
                value.display(string + "\t")


def getArchive(dataSetName):
    """ Checks if a specific dataset is present in the local directory, if not,
    downloads it.

    Args:
        dataSetName (str): the dataset name
    """
    datasets_url = {
        "Car": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab2/datasets/car",
        "Chess": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab2/datasets/chess",
        "Tennis": "https://raw.githubusercontent.com/cs-pub-ro/ML/master/lab2/datasets/tennis"
    }

    assert dataSetName in datasets_url
  
    dataset_url = datasets_url[dataSetName]
    dataset_file = dataset_url.split("/")[-1]

    from os import path
    if not path.isfile(dataset_file):
        import urllib
        print("Downloading...")
        urllib.request.urlretrieve(dataset_url, filename=dataset_file)
        assert(path.isfile(dataset_file))
        print("Got the archive")
    else:
        print(f"{dataset_file} already in the local directory")


def getDataSet(dataSetName):
    """ Reads a dataset

    Args:
        dataSetName (str): Name for the dataset

    Returns:
        A tuple containing (classes, attributes, examples):
        classes (set): the classes that are found in the dataset
        attributes (list of strings): the attributes for the dataset
    """

    dataset_file = dataSetName.lower()

    f_in = open(dataset_file, 'r')
    csv_reader = csv.reader(f_in, delimiter=",")

    # Read the header row
    row = next(csv_reader)

    # The last element represents the class
    attributeNames = row[:-1]
    
    examples = []
    classes = set()

    for row in csv_reader:
        *attributes, label = row
        classes.add(label)
        example = dict(zip(attributeNames, attributes))
        example["CLASS"] = label
        examples.append(example)
    
    f_in.close()
    return classes, attributes, examples

### Descărcare și încarcare set de date

In [0]:
getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)

## 6. Cerințe

1. [3 pct] Implementați o funcție recursivă *randomTree* care construiește arbori de decizie de adâncime **d** pe baza unui **set de date X** și a unei **mulțimi de atribute A** astfel:
 * Dac̆a *d = 0*, atunci se construies,te un nod frunz ̆a cu clasa majoritar ̆a din X.
 * Dacă *d > 0*, atunci se alege aleator un atribut $a_i$ din A și se construiește câte un subarbore pentru fiecare valoare $v_j$ a atributului $a_i$ apelând *randomTree* pentru *d − 1*:
$$
X_{i/j} = \{x \in X|a_{i}(x) = v_k\}\\
A_{new} = A \setminus \{a_i\}
$$

In [0]:
def randomTree(d, X, A):
    # Cerință 1
    pass

2. [3 pct] Implementați o funcție recursivă *id3* care construiește arbori de decizie pe baza unui **set de date X** și a unei **mulțimi de atribute A**.
    Trebuie avute în vedere următoarele aspecte:
  * dacă toate exemplele din X aparțin unei singure clase C, atunci se construiește un nod frunză etichetat cu acea clasă C
  * dacă nu mai exista atribute, atunci construiește nodul frunză etichetat cu cea mai frecventă clasă din X
    
  În caz contrar:
  * se alege atributul $a^*$ care aduce cel mai mai mare câștig informațional și se construiește un *nod intermediar* corespunzător acestuia.
$$
entropy(X) = -\sum_{c \in C}\frac{|X_c|}{|X|}log_2\frac{|X_c|}{|X|}
\\
gain(X, a) = entropy(X) - \sum_{v_{j} \in vals(a)} \frac{|X_{i/j}|}{|X|}entropy(X_{i/j})
\\
a^* = \underset{a \in A}{\operatorname{arg max}}\ gain(X, a)
$$
  * pentru fiecare valoare posibilă $v_j$ a lui $a^*$ se construiește un subarbore apelând recursiv funcția *id3* pentru:

$$
  X_j = \{x \in X|a^*(x) = v_j\}\\
  A_{new} = A\setminus\{a^*\}
$$

În cazul prezentat mai sus, entropia este utilizată pentru a măsura randomness-ul din date. Intuitiv, cu cât un eveniment are probabilitate mai mare să se întâmple atunci acesta va avea o entropia din ce în ce mai mică. Prin modul în care se construiește arborele *ID3* se încearcă reducerea entropiei alegând la fiecare pas atributele care ne ofera cea mai multă informație. Cât considerați că este entropia într-un *nod frunză*?

In [0]:
def mostFrequentClass(X):
    # TODO Cerință
    return None


def entropy(X):
    # TODO Cerință 2
    entropy = 0
    return entropy


def gain(X, A):
    # TODO Cerință 2
    gain = 0
    return gain


def id3(X, A):
    # TODO Cerință 2
    tree = Node("TODO")
    return tree

3. [4 pct] Implementați clasificator de tip pădure de arbori aleatori construind *n* arbori de adâncime maximă *d* fiecare dintre aceștia pornind de la o submulțime aleatoare a lui X.

    Folosiți funcția *randomTree* de la cerința 1.
  * Porniți de la *n = 100*, *d = 3* și submulțimi formate din 50% din elementele lui X alese la întamplare și experimentați cu acești hiperparametrii.
  * Pentru predicția clasei pentru obiecte noi alegeți clasa majoritară
  * Comparați rezultatele obținute folosind un singur arbore construit cu ID3 și o pădure de arbori aleatori. Discuție după *zgomot*, *overfitting*.

In [0]:
def randomForest(X, n, d):
    # TODO Cerință 
    pass

## 7. Set de date

În cadrul acestui laborator se vor folosi seturile de date [Car Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation), [Chess](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29) și [Tennis](https://www.kaggle.com/fredericobreno/play-tennis).

Aceste seturi de date sunt "ușor" modificate astfel încât pe prima linie să se afle atributele și labelul/clasa din care face parte fiecare exemplu.

Atributele datasetului *Chess* nu sunt intuitive, iar dacă doriți să aflați mai multe informații despre acestea, puteți accesa link-ul de [aici](https://pdfs.semanticscholar.org/db58/88d3f373aff2c6bd7b2f956b81c6896874a9.pdf?_ga=2.193733611.798337455.1582711694-486327444.1582711694).

## 8. Extra

Un exemplu mai detaliat pentru construcția arborelui de decizie ID3 se poate găsi [aici](https://github.com/cs-pub-ro/ML/blob/master/lab2/id3_example.pdf).